In [1]:
# this are all the dependencies we need
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os
import json
from api_keys import g_key
import pandas as pd

In [2]:
# let us acess gmaps with the key
gmaps.configure(api_key=g_key)

In [3]:
# we are going to bring the data from the step 1 into a dataframe
cities_dataframe = pd.read_csv("clean_cities_data.csv")
cities_dataframe.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tiksi,71.6872,128.8694,43.07,81,83,9.48,RU,1628019158
1,Baykit,61.6700,96.3700,62.28,89,76,3.96,RU,1628019158
2,Yellowknife,62.4560,-114.3525,62.56,67,90,5.99,CA,1628019069
3,Pevek,69.7008,170.3133,37.24,86,67,3.74,RU,1628019159
4,Adrar,20.5022,-10.0711,91.90,32,12,9.93,MR,1628019159


In [4]:
# we are going to get the latitude and longitude separated
coordinates = cities_dataframe[["Lat","Lng"]]

In [5]:
# get humidity
humidity = cities_dataframe["Humidity"]

In [6]:
# get temperature
maximum_temperature = cities_dataframe[["Max Temp"]]

In [7]:
# we need to create the layout and figure
layout = {
    'width': '2000px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
figure = gmaps.figure()


In [8]:
# create the heatmap layout
heatmap_layout = gmaps.heatmap_layer(coordinates, weights = humidity,
                  dissipating=False, max_intensity=300,
                  point_radius=3, opacity = 0.60)
markings = gmaps.marker_layer(coordinates)

In [9]:
# display the figure
figure.add_layer(heatmap_layout)
figure

Figure(layout=FigureLayout(height='420px'))

In [10]:
# let us reduce the dataframe to fit the following
# A max temperature lower than 80 degrees, but higher than 70
cities_dataframe_80_70 = (cities_dataframe["Max Temp"] < 80) & (cities_dataframe["Max Temp"] > 70)

In [11]:
# Wind speed less than 10 mph
cities_wind = cities_dataframe["Wind Speed"] < 10

In [12]:
# Zero cloudiness
cities_cloud = cities_dataframe.Cloudiness == 0
# see final dataframe
final_dataframe = cities_dataframe_80_70 & cities_wind & cities_cloud

In [13]:
# final touches
adjusted_dataframe = cities_dataframe[final_dataframe]
adjusted_dataframe = adjusted_dataframe.dropna()
adjusted_dataframe = adjusted_dataframe.reset_index()
adjusted_dataframe.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,21,Grand Gaube,-20.0064,57.6608,70.20,71,0,3.00,MU,1628019164
1,196,Archidona,37.0965,-4.3887,70.23,69,0,5.68,ES,1628019214
2,205,Enshi,30.3000,109.4833,73.18,96,0,1.23,CN,1628019217
3,225,Goderich,43.7501,-81.7165,77.04,60,0,8.05,CA,1628019222
4,331,San Roque,36.2107,-5.3842,74.21,83,0,5.75,ES,1628019252


In [14]:
adjusted_dataframe.columns

Index(['index', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [15]:
# we want to find the first hotel near each city within 5000 meters
hotel_dataframe_stored = adjusted_dataframe
hotel_dataframe_stored["Hotel Name"] = ""

In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
parameters = {
        "keyword": "hotel",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }

# loop that will find the location based on the coordinates given
for index, row in hotel_dataframe_stored.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    parameters["location"]=f"{lat},{lng}"

# creating a request
    hoteldata = requests.get(base_url, parameters).json()

# this will be our exception  
    try:
        hotelname= hoteldata["results"][0]["name"]
        hotel_dataframe_stored.loc[index, "Hotel Name"]= hotelname
        print(hotelname)     
    except:
        hotel_dataframe_stored.loc[index,"Hotel Name"]= np.nan
hotel_dataframe_stored.dropna(inplace=True)
# print results        
hotel_dataframe_stored

LUX Grand Gaube Resort & Villas
Hotel Escua
Moevenpick Enshi
Samuels Hotel
AC Hotel by Marriott La Linea
Palace of Zinos
LUX Grand Gaube Resort & Villas
CHERRY GARDEN CITY&SPA HOTEL
Ariane Hotel
BHR Treviso Hotel
Central City Hotel Makhachkala
Al Djadid


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,21,Grand Gaube,-20.0064,57.6608,70.20,71,0,3.00,MU,1628019164,LUX Grand Gaube Resort & Villas
1,196,Archidona,37.0965,-4.3887,70.23,69,0,5.68,ES,1628019214,Hotel Escua
2,205,Enshi,30.3000,109.4833,73.18,96,0,1.23,CN,1628019217,Moevenpick Enshi
3,225,Goderich,43.7501,-81.7165,77.04,60,0,8.05,CA,1628019222,Samuels Hotel
4,331,San Roque,36.2107,-5.3842,74.21,83,0,5.75,ES,1628019252,AC Hotel by Marriott La Linea
5,352,Ponta do Sol,32.6667,-17.1000,74.71,69,0,5.93,PT,1628019259,Palace of Zinos
6,391,Cap Malheureux,-19.9842,57.6142,70.29,70,0,3.00,MU,1628019270,LUX Grand Gaube Resort & Villas
7,466,Heraclea,37.5133,34.0467,71.80,27,0,2.21,TR,1628019292,CHERRY GARDEN CITY&SPA HOTEL
8,515,Fos-sur-Mer,43.4377,4.9446,76.84,60,0,6.91,FR,1628019316,Ariane Hotel
9,525,Quinto di Treviso,45.6481,12.1667,77.27,69,0,1.14,IT,1628019319,BHR Treviso Hotel


In [17]:
# Now we want to plot the hotels on the humidity heatmap with hotel names, city, and country

# here we add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_dataframe_stored.iterrows()]

#Create a map using coordinates to set markers
marker_locations = hotel_dataframe_stored[["Lat", "Lng"]]

In [18]:
# Add marker layer for Hotel map
figure = gmaps.figure()

markings = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)

figure.add_layer(heatmap_layout)
figure.add_layer(markings)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))